In [1]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Mounted at /content/drive


In [2]:
import os
import numpy as np
import seaborn as sns

sns.set()


In [3]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from loader_util.nn.conv import MiniGoogleNet
from loader_util.callbacks import TrainingMonitor
##
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

In [4]:
epoch_nums = 70
#init_lr = 0.001
#init_lr = 0.01
init_lr = 0.005
batch_size = 64


def poly_decay(epoch):
    power = 1.0
    alpha = init_lr * (1 - (epoch / epoch_nums)) ** power
    return alpha

In [5]:
# construct the argument parser
args = {
    "model": r"/content/drive/MyDrive/ImageDatasets/minigooglenet/model/model_lr_0.005.model",
    "output": r"/content/drive/MyDrive/ImageDatasets/minigooglenet/output"
}

In [6]:
# load the train and test data
print(f"[INFO] loading cifar10 data......")
((trainx, trainy), (testx, testy)) = cifar10.load_data()
trainx = trainx.astype("float32")
testx = testx.astype("float32")

# apply mean subtraction to the data
mean = np.mean(trainx, axis=0)
trainx = trainx - mean
testx = testx - mean

# encode the labels
lb = LabelBinarizer()
trainy = lb.fit_transform(trainy)
testy = lb.transform(testy)

[INFO] loading cifar10 data......
170500096/170498071 [==============================] - 11s 0us/step


In [7]:
# init the image generator
aug = ImageDataGenerator(width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [8]:
# construct the set callbacks
fig_path = os.path.sep.join([args["output"], f"{os.getpid()}.png"])
json_path = os.path.sep.join([args["output"], f"{os.getpid()}.png"])
callbacks = [TrainingMonitor(fig_path, jsonPath=json_path)]

In [9]:
# init the optimizer and model
print(f"[INFO] compiling model......")
opt = SGD(learning_rate=init_lr, momentum=0.9)
model = MiniGoogleNet.build(32, 32, 3, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])

[INFO] compiling model......


In [10]:
print(f"[INFO] train the net......")
model.fit_generator(aug.flow(trainx, trainy, batch_size=batch_size),
                    validation_data=(testx, testy),
                    steps_per_epoch=len(trainx) // batch_size,
                    epochs=epoch_nums,
                    callbacks=callbacks,
                    verbose=1)

[INFO] train the net......


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/70
781/781 [==============================] - 57s 50ms/step - loss: 1.4975 - accuracy: 0.4590 - val_loss: 1.3803 - val_accuracy: 0.5091
Epoch 2/70
781/781 [==============================] - 38s 49ms/step - loss: 1.1067 - accuracy: 0.6042 - val_loss: 0.9739 - val_accuracy: 0.6498
Epoch 3/70
781/781 [==============================] - 38s 49ms/step - loss: 0.9527 - accuracy: 0.6637 - val_loss: 1.0011 - val_accuracy: 0.6546
Epoch 4/70
781/781 [==============================] - 38s 49ms/step - loss: 0.8682 - accuracy: 0.6974 - val_loss: 1.2143 - val_accuracy: 0.6001
Epoch 5/70
781/781 [==============================] - 38s 49ms/step - loss: 0.7998 - accuracy: 0.7195 - val_loss: 1.0581 - val_accuracy: 0.6527
Epoch 6/70
781/781 [==============================] - 38s 49ms/step - loss: 0.7397 - accuracy: 0.7433 - val_loss: 0.7775 - val_accuracy: 0.7287
Epoch 7/70
781/781 [==============================] - 38s 49ms/step - loss: 0.7036 - accuracy: 0.7545 - val_loss: 0.9367 - val_accuracy:

In [11]:
print(f"[INFO] serialising the model......")
model.save(args["model"])

[INFO] serialising the model......
INFO:tensorflow:Assets written to: /content/drive/MyDrive/ImageDatasets/minigooglenet/model/model_lr_0.005.model/assets
